# Imports

In [3]:
import json


import ccxt
import requests


import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4582_Download_larger_universe_'
  hash='844105035'
  # Last commits:
    * 844105035 vlady    CmTask4582: fix for empty dataframes                              (  14 hours ago) Tue Jun 20 17:09:06 2023  (HEAD -> CmTask4582_Download_larger_universe_, origin/CmTask4582_Download_larger_universe_)
    * 2ea4d214f vlady    CmTask4582: new universe v7_5                                     (  18 hours ago) Tue Jun 20 12:54:02 2023           
    * 40f5f7872 vlady    CmTask4582: compare up to 200 symbols from the coinmarketcap      (  19 hours ago) Tue Jun 20 12:35:19 2023        

# Get coinmarketcap top 200

In [13]:
# To download top 100 from https://coinmarketcap.com/ we will use it's API. 
# Simple registration give us personal API KEY

CMC_PRO_API_KEY = "31eb170c-a347-4c3e-824a-76c5aaa8b22d"
LIMIT = "200"
latest_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
parameters = {
  'start':'1',
  'limit': LIMIT,
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': CMC_PRO_API_KEY,
}

In [14]:
result = requests.get(
    latest_url,
    params=parameters,
    headers=headers
)

In [15]:
data = result.json()["data"]

In [16]:
coinmarketcap_symbols = [e["symbol"] for e in data]

In [17]:
coinmarketcap_symbols = pd.Series(coinmarketcap_symbols)

# Get available binance futures currency pairs

## CCXT

In [30]:
binance_ccxt = ccxt.binance({'options': {'defaultType': 'future'}, })

In [31]:
markets = binance_ccxt.loadMarkets()

In [33]:
markets_futures = {pair: data for pair, data in markets.items() if data["future"]}

In [34]:
market_futures_currencies = [data["base"] for pair, data in markets_futures.items()]

In [78]:
ccxt_market_futures_currencies = pd.Series(list(set(market_futures_currencies)))

## Binance API

In [18]:
url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

payload = {}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

In [19]:
symbols = [symbol for symbol in response.json()["symbols"] if symbol["status"] == "TRADING"]

In [20]:
binance_api_symbols = set([s["baseAsset"] for s in symbols])

In [21]:
binnace_api_symbols = pd.Series(list(binance_api_symbols))

# Find intersections

In [22]:
len(coinmarketcap_symbols[coinmarketcap_symbols.isin(binnace_api_symbols)])

128

In [23]:
len(coinmarketcap_symbols[coinmarketcap_symbols.isin(ccxt_market_futures_currencies)])

NameError: name 'ccxt_market_futures_currencies' is not defined

In [24]:
symbols_intersection = list(coinmarketcap_symbols[coinmarketcap_symbols.isin(binnace_api_symbols)])

In [25]:
top_100_symbols_intersections = symbols_intersection[:100]

In [27]:
new_universe = [f"{symbol}_USDT" for symbol in top_100_symbols_intersections]

In [28]:
old_universe = [
        "ETH_USDT",
        "BTC_USDT",
        "SAND_USDT",
        "STORJ_USDT",
        "GMT_USDT",
        "AVAX_USDT",
        "BNB_USDT",
        "APE_USDT",
        "MATIC_USDT",
        "DYDX_USDT",
        "DOT_USDT",
        "UNFI_USDT",
        "LINK_USDT",
        "XRP_USDT",
        "CRV_USDT",
        "RUNE_USDT",
        "BAKE_USDT",
        "NEAR_USDT",
        "FTM_USDT",
        "WAVES_USDT",
        "AXS_USDT",
        "OGN_USDT",
        "DOGE_USDT",
        "SOL_USDT",
        "CTK_USDT"
   ]

In [29]:
new_mixed_universe = list(set(new_universe + old_universe))

In [31]:
len(new_mixed_universe)

106

# Read and check downloaded data

In [8]:
signature = "bulk.airflow.downloaded_1min.parquet.ohlcv.futures.v7_5.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-02-01T00:00:00+00:00")
end_timestamp = pd.Timestamp("2023-06-01T00:00:00+00:00")
binance_ohlcv_data = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [9]:
binance_ohlcv_data.head()

,timestamp,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,
2023-02-01 00:00:00+00:00,1675209600000,0.5141,0.5143,0.5137,0.5142,59009.0,binance,2023-06-20 17:19:02.877891+00:00,1INCH_USDT,2023,2
2023-02-01 00:01:00+00:00,1675209660000,0.5142,0.5143,0.5140,0.5140,26966.0,binance,2023-06-20 17:19:02.877891+00:00,1INCH_USDT,2023,2
2023-02-01 00:02:00+00:00,1675209720000,0.5141,0.5157,0.5141,0.5154,49452.0,binance,2023-06-20 17:19:02.877891+00:00,1INCH_USDT,2023,2
2023-02-01 00:03:00+00:00,1675209780000,0.5154,0.5162,0.5153,0.5157,60371.0,binance,2023-06-20 17:19:02.877891+00:00,1INCH_USDT,2023,2
2023-02-01 00:04:00+00:00,1675209840000,0.5155,0.5157,0.5147,0.5148,24003.0,binance,2023-06-20 17:19:02.877891+00:00,1INCH_USDT,2023,2


In [10]:
downloaded_currency_pairs = pd.Series(binance_ohlcv_data.currency_pair.unique())

In [45]:
downloaded_currency_pairs.shape

(105,)